# Subsequent Memory Effect

### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [2]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [9]:
# load spectral parameterization results
df = pd.read_csv(f"{PROJECT_PATH}/data/results/spectral_parameters.csv", index_col=0)

# log transform power values
for var in ['alpha_bp','alpha_adj']:
    df[var] = df[var].apply(np.log10)
df['alpha_adj'].loc[np.isnan(df['alpha_adj'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)

# set Nan values to zero (no detectable alpha peak)
df['alpha_pw'].loc[np.isnan(df['alpha_pw'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)

# show
print(df.shape)
print(len(df)/8)
df.head()

(5560, 19)
695.0


C:\Users\micha\AppData\Local\Temp\ipykernel_23896\2329723685.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alpha_adj'].loc[np.isnan(df['alpha_adj'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)
C:\Users\micha\AppData\Local\Temp\ipykernel_23896\2329723685.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alpha_pw'].loc[np.isnan(df['alpha_pw'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,offset,knee,exponent,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,prestim,8.451107,12.903272,3.542673,10.978558,NaN,0.000000,NaN,4.443699,3.753137
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,prestim,7.595982,12.332651,3.052480,83.836257,10.839096,0.210785,2.000000,4.159722,3.469575
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,prestim,7.056773,8.787521,2.774045,89.824561,12.850783,0.276397,2.000000,4.178573,3.630406
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,prestim,7.595080,10.177476,2.940543,59.883041,NaN,0.000000,NaN,4.374244,3.435538
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,prestim,7.399935,9.126477,2.872949,65.871345,11.565201,0.227912,2.671217,4.337301,3.551039


In [10]:
# compute stimulus-evoked change in each parameter
df_pre = df.loc[df['epoch']=='prestim']
df_post = df.loc[df['epoch']=='poststim']

# compute change
columns = df_pre.columns.tolist()
df_diff = df_pre.loc[:, columns[:10]]
df_diff.loc[:, columns[10:]] = df_post.loc[:, columns[10:]].values - df_pre.loc[:, columns[10:]].values
df_diff.loc[:,'f_rotation'] = df_pre.loc[:,'f_rotation']
df_diff.loc[:,'alpha_bp'] = df_post.loc[:,'alpha_bp'].values - df_pre.loc[:,'alpha_bp'].values
df_diff.loc[:,'alpha_adj'] = df_post.loc[:,'alpha_adj'].values - df_pre.loc[:,'alpha_adj'].values
df_diff

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,offset,knee,exponent,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,prestim,-0.141194,-1.483239,-0.037759,10.978558,NaN,0.000000,NaN,0.009420,-0.039453
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,prestim,-0.421704,-5.132790,-0.218467,83.836257,NaN,-0.210785,NaN,0.072515,-0.196096
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,prestim,-0.094881,-0.694436,-0.048501,89.824561,NaN,-0.276397,NaN,-0.136907,-1.030992
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,prestim,-0.631166,-2.501609,-0.354102,59.883041,NaN,0.213641,NaN,-0.062359,0.282614
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,prestim,-0.575058,-2.853818,-0.315757,65.871345,NaN,-0.227912,NaN,-0.217816,-3.551039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4860,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,faces,miss,prestim,-0.095198,-1.291748,-0.058494,40.920078,NaN,0.000000,NaN,-0.004656,0.031943
4861,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,faces,miss,prestim,-0.345755,-3.174832,-0.224530,31.937622,NaN,0.000000,NaN,0.068206,0.000000
4862,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,faces,miss,prestim,-0.109624,-3.361556,-0.036085,17.964912,NaN,0.000000,NaN,0.131772,0.000000
4863,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,faces,miss,prestim,-0.346605,-7.055522,-0.206374,43.914230,NaN,0.000000,NaN,0.242971,0.000000


In [11]:
# marge with task-modulation results

# load task-modulation results
df_tm = pd.read_csv(r"C:\Users\micha\projects\oscillation_vs_exponent\data\results\ieeg_modulated_channels.csv", index_col=0)
df_tm['sig'] = np.where(df_tm['pval'] < 0.05, 1, 0) # determine significance

# join
df_merge = pd.merge(df_diff, df_tm, on=['patient','chan_idx','material','memory'])
df_merge

df_merge

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,...,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj,pval_uncorrected,sign,pval,sig,sig_tm
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,prestim,...,NaN,0.000000,NaN,0.009420,-0.039453,0.0360,1.0,0.0360,1,True
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,prestim,...,NaN,-0.210785,NaN,0.072515,-0.196096,0.0280,1.0,0.0280,1,True
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,prestim,...,NaN,-0.276397,NaN,-0.136907,-1.030992,0.0002,1.0,0.0002,1,True
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,prestim,...,NaN,0.213641,NaN,-0.062359,0.282614,0.0014,1.0,0.0014,1,True
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,prestim,...,NaN,-0.227912,NaN,-0.217816,-3.551039,0.0000,1.0,0.0000,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,faces,miss,prestim,...,NaN,0.000000,NaN,-0.004656,0.031943,0.1656,-1.0,0.1656,0,False
2776,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,faces,miss,prestim,...,NaN,0.000000,NaN,0.068206,0.000000,0.9758,-1.0,0.9758,0,False
2777,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,faces,miss,prestim,...,NaN,0.000000,NaN,0.131772,0.000000,0.0140,-1.0,0.0140,1,True
2778,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,faces,miss,prestim,...,NaN,0.000000,NaN,0.242971,0.000000,0.0368,-1.0,0.0368,1,True


### Run 2x2 Anova

In [12]:
# find channels that are significant across material conditions
df_hit = df_merge.loc[df_merge['memory']=='hit']
sig = df_hit.groupby(['patient','chan_idx']).all().reset_index()

# add to df_merge
df_merge['task_modulated'] = np.nan
for ii in range(len(sig)):
    df_merge.loc[(df_merge['patient']==sig.loc[ii, 'patient']) & \
                (df_merge['chan_idx']==sig.loc[ii, 'chan_idx']), 'task_modulated'] \
                    = sig.loc[ii, 'sig']
    
# reduce
df_tm = df_merge.loc[df_merge['task_modulated']]

# show
print(f"N task-modulated channels: \t{int(len(df_tm)/4)}")
df_tm



N task-modulated channels: 	145


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,...,alpha_pw,alpha_bw,alpha_bp,alpha_adj,pval_uncorrected,sign,pval,sig,sig_tm,task_modulated
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,prestim,...,0.000000,NaN,0.009420,-0.039453,0.0360,1.0,0.0360,1,True,True
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,prestim,...,-0.210785,NaN,0.072515,-0.196096,0.0280,1.0,0.0280,1,True,True
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,prestim,...,-0.276397,NaN,-0.136907,-1.030992,0.0002,1.0,0.0002,1,True,True
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,prestim,...,-0.227912,NaN,-0.217816,-3.551039,0.0000,1.0,0.0000,1,True,True
9,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,pat04_0,words,hit,prestim,...,-0.177757,-0.195025,-0.430993,-0.545363,0.0000,1.0,0.0000,1,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,pat22,25,PI_05-PI_06,-10.50000,-40.00000,32.50000,pat22_25,faces,miss,prestim,...,0.441313,NaN,0.151758,0.222912,0.0302,-1.0,0.0302,1,True,True
2753,pat22,26,PI_06-PI_07,-11.50000,-40.00000,39.00000,pat22_26,faces,miss,prestim,...,0.000000,NaN,0.132706,2.586917,0.1644,-1.0,0.1644,0,False,True
2756,pat22,29,PI_09-PI_10,-11.50000,-41.00000,60.50000,pat22_29,faces,miss,prestim,...,0.000000,NaN,0.069439,-3.116804,0.1330,-1.0,0.1330,0,False,True
2763,pat22,36,AC_07-AC_08,23.50000,-31.00000,42.00000,pat22_36,faces,miss,prestim,...,0.000000,NaN,0.185454,0.000000,0.0052,-1.0,0.0052,1,True,True


In [13]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exponent', 'alpha_pw', 'alpha_bp', 'alpha_adj']:
    # repeated measures anova
    results_i = sm.stats.AnovaRM(df_tm, f"{var}", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exponent  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           5.1089 1.0000 144.0000 0.0253
material         6.9942 1.0000 144.0000 0.0091
memory:material  0.6597 1.0000 144.0000 0.4180



===================  alpha_pw  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.2451 1.0000 144.0000 0.6213
material         0.1794 1.0000 144.0000 0.6726
memory:material  0.0943 1.0000 144.0000 0.7592



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.7329 1.0000 144.0000 0.3934
material         1.9479 1.0000 144.0000 0.1650
memory:material  0.0004 1.0000 144.0000 0.9846



===================  alpha_adj  ==================

In [16]:
df_mean = df.groupby(by=['material','memory']).mean()
df_mean.loc[:, ['exponent', 'alpha_pw', 'alpha_bp', 'alpha_adj']]

exponent  alpha_pw  alpha_bp  alpha_adj
material memory                                         
faces    hit     3.003687  0.168712  3.797294   2.103949
         miss    3.015558  0.161760  3.799909   2.088484
words    hit     3.006810  0.163378  3.808979   2.073471
         miss    3.034050  0.172801  3.828585   2.275645